# Summary

This notebook runs all aspects of the Regression STREAMLINE which is an automated machine learning analysis pipeline for regression tasks. Of note, two potentially important elements that are not automated by this pipeline include careful data cleaning and feature engineering using problem domain knowledge. Please review the README included in the associated GitHub repository for a detailed overview of how to run this pipeline. For simplicity, this notebook runs Python code outside of what is visible within it.

## Google Collab and Run Enviornment Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Load all require local python files on from Google Drive
from google.colab import files

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/ExploratoryAnalysisMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/ExploratoryAnalysisJob.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/DataPreprocessingMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/DataPreprocessingJob.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/FeatureImportanceMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/FeatureImportanceJob.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/FeatureSelectionMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/FeatureSelectionJob.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/ModelMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/ModelJob.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/l21regjob.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/smogn.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/StatsMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/StatsJob.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/DataCompareMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/DataCompareJob.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/PDF_ReportMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/PDF_ReportJob.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/PDF_ReportJob_Reg.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/ApplyModelMain.py /content
!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/ApplyModelJob.py /content

!cp /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/streamline/FileCleanup.py /content

In [3]:
#Install remaining required packages not preinstalled in Google Collab
!pip install skrebate==0.7
!pip install xgboost
!pip install lightgbm
!pip install catboost
!pip install gplearn
!pip install scikit-eLCS
!pip install scikit-XCS
!pip install scikit-ExSTraCS
!pip install optuna==2.0.0
!pip install plotly
!pip install kaleido==0.0.3.post1
!pip install fpdf
!pip install group-lasso

  Preparing metadata (setup.py) ... done
  Created wheel for skrebate: filename=skrebate-0.7-py3-none-any.whl size=30620 sha256=372ce64e664fd585d1b7ed04d85166468ef193effd43772213f324c0b192783d
  Stored in directory: /root/.cache/pip/wheels/3f/c9/40/5911f5f510e1252264e84984b560446d8e63241b709d82f7d5
Successfully built skrebate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-eLCS: filename=scikit_eLCS-1.2.4-py3-none-any.whl size=38958 sha256=da230fd78bfd42b58ebc6d240b9467d59f5ab83f441401f491a506f9cfe3b97c
  Stored in directory: /root/.cache/pip/wheels/0a/9d/e2/a17d6ce42dc869399bf148eeb8cfb231890b75f1dd2f63ac0c
Successfully built scikit-eLCS
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.6/317.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-XCS: filename=scikit_XC

## Notebook Housekeeping
Set up notebook cells to display desired results. No need to edit.

In [4]:
import warnings
import sys
import os
import shutil
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## -----------------------------------------------------------------------------------------------------------------
## (User Specified) Run Parameters of STREAMLINE
These initial notebook cells include all customizable run parameters for STREAMLINE. These settings should only be left unchanged for users wishing to test out the pipeline demo (as is) to learn how it works or to confirm efficacy before running their own data. Run parameters for each phase of the pipeline are included in separate code cells of this section of the notebook.


### Mandatory Run Parameters for Pipeline

In [5]:
demo_run = False #Leave true to run the local demo dataset (without specifying any datapaths), make False to specify a different data folder path below

#Target dataset folder path(must include one or more .txt or .csv datasets)
data_path = "/content/drive/MyDrive/STREAMLINE-Regression/Measurements/cdrsb" # (str) Demontration Data Path Folder

#Output foder path: where to save pipeline outputs (must be updated for a given user)
output_path = '/content/drive/MyDrive/STREAMLINE-Regression/Colab_Output' # (str) Demonstration Ouput Path Folder

#Unique experiment name - folder created for this analysis within output folder path
experiment_name = 'cdrsb_experiment'  # (str) Demontration Experiment Name

# Data Labels
class_label = 'Cognition_Score' # (str) i.e. class outcome column label
instance_label = 'Class' # (str) If data includes instance labels, given respective column name here, otherwise put 'None'

#Option to manually specify feature names to leave out of analysis, or which to treat as categorical (without using built in variable type detector)
ignore_features = [] # list of column names (given as string values) to exclude from the analysis (only insert column names if needed, otherwise leave empty)
categorical_feature_headers = [] # empty list for 'auto-detect' otherwise list feature names (given as string values) to be treated as categorical. Only impacts algorithms that can take variable type into account.

### Run Parameters for Phase 1: Exploratory Analysis

In [6]:
cv_partitions = 5  # (int, > 1) Number of training/testing data partitions to create - and resulting number of models generated using each ML algorithm
partition_method = 'R' # (str, S R or M) for stratified, random, or matched, respectively
match_label = 'None' # (str) Only applies when M selected for partition-method; indicates column label with matched instance ids'

categorical_cutoff = 10 # (int) Number of unique values after which a variable is considered to be quantitative vs categorical
sig_cutoff = 0.05 # (float, 0-1) Significance cutoff used throughout pipeline
export_feature_correlations = 'True' # (str, True or False) Run and export feature correlation analysis (yields correlation heatmap)
export_univariate_plots = 'False' # (str, True or False) Export univariate analysis plots (note: univariate analysis still output by default)
topFeatures = 20 # (int) Number of top features to report in notebook for univariate analysis
random_state = 42 # (int) Sets a specific random seed for reproducible results

### Run Parameters for Phase 2: Data Preprocessing

In [7]:
scale_data = 'True' # (str, True or False) Perform data scaling?
impute_data = 'True' # (str, True or False) Perform missing value data imputation? (required for most ML algorithms if missing data is present)
overwrite_cv = 'True' # (str, True or False) Overwrites earlier cv datasets with new scaled/imputed ones
multi_impute = 'True' # (str, True or False) Applies multivariate imputation to quantitative features, otherwise uses mean imputation

### Run Parameters for Phase 3: Feature Importance Evaluation

In [8]:
do_mutual_info = 'True' # (str, True or False) Do mutual information analysis
do_multisurf = 'True' # (str, True or False) Do multiSURF analysis
use_TURF = 'False' # (str, True or False) Use TURF wrapper around MultiSURF
TURF_pct = 0.5 # (float, 0.01-0.5) Proportion of instances removed in an iteration (also dictates number of iterations)
njobs = -1 # (int) Number of cores dedicated to running algorithm; setting to -1 will use all available cores
instance_subset = 2000 # (int) Sample subset size to use with multiSURF

### Run Parameters for Phase 4: Feature Selection

In [9]:
max_features_to_keep = 2000 # (int) Maximum features to keep.
filter_poor_features = 'False' # (str, True or False) Filter out the worst performing features prior to modeling
top_features = 40 # (int) Number of top features to illustrate in figures
export_scores = 'True' # (str, True or False) Export figure summarizing average feature importance scores over cv partitions

### Run Parameters for Phase 5: Modeling

In [10]:
#ML Model Algorithm Options (individual hyperparameter options can be adjusted below)
do_all = 'False'
# Regression Algorithm
do_CommonReg = 'True'
do_linReg = 'True'
do_ENReg = 'True'
do_RFReg = 'False'
do_AdaReg = 'False'
do_GradReg = 'False'
do_SVR = 'True'
do_GL = 'False'

# L21 Regression Series Algorithm
do_L21Series = 'True'
do_L21Reg = 'True'
do_L21GMMReg = 'False'
do_L21DGMMReg = 'False'

#Group Lasso Parameters - Defaults available
groups_path = '/content/drive/MyDrive/STREAMLINE-Regression/streamline/groups.csv' # (str) Path of the defined groups

# Other Analysis Parameters
training_subsample = 0  # (int) For long running algorithms, option to subsample training set (0 for no subsample) Limit Sample Size Used to train algorithms that do not scale up well in large instance spaces (i.e. XGB,SVM,KN,ANN,and LR to a lesser degree) and depending on 'instances' settings, ExSTraCS, eLCS, and XCS)
use_uniform_FI = 'True' # (str, True or False) Overides use of any available feature importances estimate methods from models, instead using permutation_importance uniformly
primary_metric = 'explained_variance' # (str) Must be an available metric identifier from (https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

# Hyperparameter Sweep Options
n_trials = 50   # (int or None) Number of bayesian hyperparameter optimization trials using optuna
timeout = 900    # (int or None) Seconds until hyperparameter sweep stops running new trials (Note: it may run longer to finish last trial started)
export_hyper_sweep_plots = 'True' # (str, True or False) Export hyper parameter sweep plots from optuna

### Hyperparameter Sweep Options for ML Algorithms
Users can extend or limit the range or options for given ML algorithm hyperparameters to be tested in hyperparameter optimization. These options are hardcoded when running this pipeline from the command line, but they are available here for users to see and modify. We have sought to include a broad range of relevant configurations based on online examples and relevant research publications. Use caution when modifying values below as improper modifications will lead to pipeline errors/failure. Links to available hyperparameter options for each algorithm are included below.

In [11]:
def hyperparameters(random_state,feature_names):
    param_grid = {}

    # DGMM Regressor
    # https://github.com/shaoweinuaa/DGMM
    param_grid_L21Reg = {'lambda1':[1,100], 'max_iter': [10, 2500]}

    param_grid_L21GMMReg = {'lambda1':[1, 300], 'lambda2':[1, 300], 'max_iter': [10, 2500]}

    param_grid_L21DGMMReg = {'lambda1': [0.01, 100], 'lambda2': [0.01, 100], 'max_iter': [10, 2500]}

    # Elastic Net Regressor
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet
    param_grid_EN = {'alpha':[1e-3,1],'max_iter': [10,2500],
                     'l1_ratio':[0,1],'random_state':[random_state]}

    # Random Forest Regressor
    # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
    param_grid_RF = {'n_estimators': [10, 1000],'max_depth': [1, 30],'min_samples_split': [2, 50],
                     'min_samples_leaf': [1, 50],'max_features': [None, 'auto', 'log2'],
                     'bootstrap': [True],'oob_score': [False, True],'random_state':[random_state]}

    # AdaBoost Regressor
    # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html
    param_grid_AdaB = {'n_estimators': [10, 1000], 'learning_rate': [.0001, 0.3], 'loss': ['linear', 'square', 'exponential']}

    # GradientBoosting Regressor
    # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
    param_grid_GradB = {'learning_rate': [.0001, 0.3],'n_estimators': [10, 1000],
                     'min_samples_leaf': [1, 50],'min_samples_split': [2, 50], 'max_depth': [1, 30],
                     'random_state':[random_state]}

    # Epsilon-Support Vector Regression
    # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
    param_grid_SVR = {'kernel': ['linear', 'poly', 'rbf'],'C': [0.1, 1000],'gamma': ['scale'],'degree': [1, 6]}

    # Group Lasso Regressor
    # https://group-lasso.readthedocs.io/en/latest/api_reference.html#
    param_grid_GL = {'group_reg':[1e-3,1],#'l1_reg':[0,1],
                     'n_iter':[10,2500],
                     'scale_reg': ['group_size', 'none', 'inverse_group_size'],
                     #'subsampling_scheme': [0.1,0.9],
                     #'frobenius_lipschitz': [True],
                     'random_state':[random_state]}

    #Leave code below as is...
    param_grid['L21Reg'] = param_grid_L21Reg
    param_grid['L21GMMReg'] = param_grid_L21GMMReg
    param_grid['L21DGMMReg'] = param_grid_L21DGMMReg
    param_grid['Linear Regression'] = {}
    param_grid['Elastic Net'] = param_grid_EN
    param_grid['Group Lasso'] = param_grid_GL
    param_grid['RF Regressor'] = param_grid_RF
    param_grid['AdaBoost'] = param_grid_AdaB
    param_grid['GradBoost'] = param_grid_GradB
    param_grid['SVR'] = param_grid_SVR
    return param_grid

### Run Parameters for Phase 6:  Statistics Summary and Figure Generation

In [12]:
plot_FI_box = 'True' # (str, True or False) Plot box plot summaries comparing algorithms for each metric
plot_metric_boxplots = 'True' # (str, True or False) Plot feature importance boxplots for each algorithm
metric_weight = 'explained_variance' # (str, balanced_accuracy or roc_auc) ML model metric used as weight in composite FI plots (only supports balanced_accuracy or roc_auc as options) Recommend setting the same as primary_metric if possible.
top_model_features = 40  # (int) Number of top features in model to illustrate in figures

### Run Parameters for Phase 10:  Apply Models to Replication Dataset
An optional phase to apply all trained models from previous phases to a separate 'replication' dataset which will be used to evaluate models across all algorithms and CV splits. In this demo, we didn't have a separate replication dataset to use for the UCI HCC dataset evaluated. Thus here we use a copy of the original HCC dataset as a 'pretend' replication dataset to demonstrate functionality. The replication data folder can include 1 or more datasets that can be evaluated as separate replication data. The user also needs to

In [13]:
applyToReplication = False # (Boolean, True or False) Leave false unless you have a replication dataset handy to further evaluate/compare all models in uniform manner
rep_data_path = "/content/drive/MyDrive/STREAMLINE-main/DemoRepData" # (txt) Name of folder with replication Dataset(s)
dataset_for_rep = "/content/drive/MyDrive/STREAMLINE-main/DemoRepData/hcc-data_example_rep.csv" # (txt) Path and name of dataset used to generate the models we want to apply (not the replication dataset)

### Run Parameters for Phase 11:  File Cleanup
An optional phase to delete all unnecessary/temporary files generated by the pipeline.

In [14]:
del_time = 'True'  # (str, True or False) Delete individual run-time files (but save summary)
del_oldCV = 'True' # (str, True or False) Delete any of the older versions of CV training and testing datasets not overwritten (preserves final training and testing datasets)

## -----------------------------------------------------------------------------------------------------------------
## Phase 1: Exploratory Analysis

### Identify Working Directory

In [15]:
wd_path = os.getcwd() #Working directory path automatically detected
wd_path = wd_path.replace('\\','/')
sys.path.insert(1, wd_path+'/streamline')

### Import Python Packages

In [16]:
import glob
import time
import csv
import pandas as pd
import numpy as np
import random
import pickle
import ExploratoryAnalysisMain
import ExploratoryAnalysisJob

### Demo Setup
Bypasses whatever user may have entered into 'data_path' variable to ensure proper loading of local 'demo' dataset.

In [17]:
if demo_run:
    data_path = wd_path+'/drive/MyDrive/Master_Thesis/STREAMLINE-main/DemoData'
print("Data Folder Path: "+data_path)
jupyterRun = 'True' #Leave True or pipeline will not display text or figures

Data Folder Path: /content/drive/MyDrive/Master_Thesis/STREAMLINE-Regression/Measurements/cdrsb


### Imbalance Check (optional)

In [18]:
imbalance_check = 'True' #Whether to perform imbalance check or not. Default set to 'False'

### Run Exploratory Analysis

In [19]:
ExploratoryAnalysisMain.makeDirTree(data_path,output_path,experiment_name,jupyterRun)

Exception: ignored

In [ ]:
#Determine file extension of datasets in target folder:
file_count = 0
unique_datanames = []
for dataset_path in glob.glob(data_path+'/*'):
    dataset_path = str(dataset_path).replace('\\','/')
    print('---------------------------------------------------------------------------------')
    print(dataset_path)
    file_extension = dataset_path.split('/')[-1].split('.')[-1]
    data_name = dataset_path.split('/')[-1].split('.')[0] #Save unique dataset names so that analysis is run only once if there is both a .txt and .csv version of dataset with same name.
    if file_extension == 'txt' or file_extension == 'csv':
        if data_name not in unique_datanames:
            unique_datanames.append(data_name)
            ExploratoryAnalysisJob.runExplore(dataset_path,output_path+'/'+experiment_name,cv_partitions,partition_method,categorical_cutoff,export_feature_correlations,export_univariate_plots,class_label,instance_label,match_label,random_state,ignore_features,categorical_feature_headers,sig_cutoff,jupyterRun)
            file_count += 1

if file_count == 0: #Check that there was at least 1 dataset
    raise Exception("There must be at least one .txt or .csv dataset in data_path directory")

#Create metadata dictionary object to keep track of pipeline run paramaters throughout phases
metadata = {}
metadata['Data Path'] = data_path
metadata['Output Path'] = output_path
metadata['Experiment Name'] = experiment_name
metadata['Class Label'] = class_label
metadata['Instance Label'] = instance_label
metadata['Ignored Features'] = ignore_features
metadata['Specified Categorical Features'] = categorical_feature_headers
metadata['CV Partitions'] = cv_partitions
metadata['Partition Method'] = partition_method
metadata['Match Label'] = match_label
metadata['Categorical Cutoff'] = categorical_cutoff
metadata['Statistical Significance Cutoff'] = sig_cutoff
metadata['Export Feature Correlations'] = export_feature_correlations
metadata['Export Univariate Plots'] = export_univariate_plots
metadata['Random Seed'] = random_state
metadata['Run From Jupyter Notebook'] = jupyterRun
#Pickle the metadata for future use
pickle_out = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'wb')
pickle.dump(metadata,pickle_out)
pickle_out.close()

## -----------------------------------------------------------------------------------------------------------------
## Phase 2: Data Preprocessing

### Import Additional Python Packages

In [ ]:
import DataPreprocessingJob

### Run Data Preprocessing

In [ ]:
dataset_paths = os.listdir(output_path+"/"+experiment_name)
dataset_paths.remove('metadata.pickle')
for dataset_directory_path in dataset_paths:
    full_path = output_path+"/"+experiment_name+"/"+dataset_directory_path
    print(dataset_directory_path)
    if dataset_directory_path != 'L21':
        for cv_train_path in glob.glob(full_path+"/CVDatasets/*Train.csv"):
            cv_train_path = str(cv_train_path).replace('\\','/')
            cv_test_path = cv_train_path.replace("Train.csv","Test.csv")
            DataPreprocessingJob.job(cv_train_path,cv_test_path,output_path+'/'+experiment_name,scale_data,impute_data,overwrite_cv,categorical_cutoff,class_label,instance_label,random_state,multi_impute,jupyterRun)


#Unpickle metadata from previous phase
file = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()

#Update metadata
metadata['Use Data Scaling'] = scale_data
metadata['Use Data Imputation'] = impute_data
metadata['Use Multivariate Imputation'] = multi_impute
#Pickle the metadata for future use
pickle_out = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'wb')
pickle.dump(metadata,pickle_out)
pickle_out.close()

## -----------------------------------------------------------------------------------------------------------------
## Phase 3: Feature Importance Evaluation

### Import Additional Python Packages

In [ ]:
import FeatureImportanceJob

### Run Feature Importance Evaluation

In [ ]:
dataset_paths = os.listdir(output_path+"/"+experiment_name)
removeList = removeList = ['metadata.pickle','metadata.csv','algInfo.pickle','jobsCompleted','logs','jobs','DatasetComparisons','UsefulNotebooks',experiment_name+'_ML_Pipeline_Report.pdf']
for text in removeList:
    if text in dataset_paths:
        dataset_paths.remove(text)

for dataset_directory_path in dataset_paths:
    full_path = output_path+"/"+experiment_name+"/"+dataset_directory_path
    experiment_path = output_path+'/'+experiment_name

    if eval(do_mutual_info) or eval(do_multisurf):
        if not os.path.exists(full_path+"/feature_selection"):
            os.mkdir(full_path+"/feature_selection")

    if eval(do_mutual_info):
        if not os.path.exists(full_path+"/feature_selection/mutualinformation"):
            os.mkdir(full_path+"/feature_selection/mutualinformation")
        for cv_train_path in glob.glob(full_path+"/CVDatasets/*_CV_*Train.csv"):
            cv_train_path = str(cv_train_path).replace('\\','/')
            FeatureImportanceJob.job(cv_train_path,experiment_path,random_state,class_label,instance_label,instance_subset,'mi',njobs,use_TURF,TURF_pct,jupyterRun)

    if eval(do_multisurf):
        if not os.path.exists(full_path+"/feature_selection/multisurf"):
            os.mkdir(full_path+"/feature_selection/multisurf")
        for cv_train_path in glob.glob(full_path+"/CVDatasets/*_CV_*Train.csv"):
            cv_train_path = str(cv_train_path).replace('\\','/')
            FeatureImportanceJob.job(cv_train_path,experiment_path,random_state,class_label,instance_label,instance_subset,'ms',njobs,use_TURF,TURF_pct,jupyterRun)

#Unpickle metadata from previous phase
file = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()

#Update metadata
metadata['Use Mutual Information'] = do_mutual_info
metadata['Use MultiSURF'] = do_multisurf
metadata['Use TURF'] = use_TURF
metadata['TURF Cutoff'] = TURF_pct
metadata['MultiSURF Instance Subset'] = instance_subset
#Pickle the metadata for future use
pickle_out = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'wb')
pickle.dump(metadata,pickle_out)
pickle_out.close()

In [ ]:
if do_L21Series == 'True':
  path_1 = output_path + "/" + experiment_name
  os.mkdir(path_1+'/L21')
  for dataset_directory_path_1 in dataset_paths:
      dest_dir = path_1+'/L21'+'/'+dataset_directory_path_1
      src_dir = path_1+'/'+dataset_directory_path_1
      shutil.copytree(src_dir, dest_dir)

## -----------------------------------------------------------------------------------------------------------------
## Phase 4: Feature Selection

### Import Additional Python Packages

In [ ]:
import FeatureSelectionJob

### Run Feature Selection

In [ ]:
dataset_paths = os.listdir(output_path + "/" + experiment_name)
removeList = removeList = ['L21', 'metadata.pickle','metadata.csv','algInfo.pickle','jobsCompleted','logs','jobs','DatasetComparisons','UsefulNotebooks',experiment_name+'_ML_Pipeline_Report.pdf']
for text in removeList:
    if text in dataset_paths:
        dataset_paths.remove(text)

for dataset_directory_path in dataset_paths:
    full_path = output_path + "/" + experiment_name + "/" + dataset_directory_path
    FeatureSelectionJob.job(full_path,do_mutual_info,do_multisurf,max_features_to_keep,filter_poor_features,top_features,export_scores,class_label,instance_label,cv_partitions,overwrite_cv,jupyterRun)

#Unpickle metadata from previous phase
file = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()

#Update metadata
metadata['Max Features to Keep'] = max_features_to_keep
metadata['Filter Poor Features'] = filter_poor_features
metadata['Top Features to Display'] = top_features
metadata['Export Feature Importance Plot'] = export_scores
metadata['Overwrite CV Datasets'] = overwrite_cv
#Pickle the metadata for future use
pickle_out = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'wb')
pickle.dump(metadata,pickle_out)
pickle_out.close()

## -----------------------------------------------------------------------------------------------------------------
## Phase 5: ML Modeling

### Phase 5 Import Additional Python Packages

In [ ]:
import ModelJob

In [ ]:
#Create ML modeling algorithm information dictionary, given as ['algorithm used (set to true initially by default)','algorithm abreviation', 'color used for algorithm on figures']
### Note that other named colors used by matplotlib can be found here: https://matplotlib.org/3.5.0/_images/sphx_glr_named_colors_003.png
### Make sure new ML algorithm abbreviations and color designations are unique
algInfo = {}

algInfo['Linear Regression'] = [True,'Linear Regression','red']
algInfo['Elastic Net'] = [True, 'Elastic Net', 'steelblue']
algInfo['Group Lasso'] = [True, 'Group Lasso', 'orange']
algInfo['L21Reg'] = [True,'L21Reg','green']
algInfo['L21GMMReg'] = [True, 'L21GMMReg', 'darkslategray']
algInfo['L21DGMMReg'] = [True, 'L21DGMMReg', 'magenta']

algInfo['RF Regressor'] = [True, 'RF Regressor', 'navy']
algInfo['AdaBoost'] = [True, 'AdaBoost', 'teal']
algInfo['GradBoost'] = [True, 'GradBoost', 'olive']
algInfo['SVR'] = [True, 'SVR', 'rosybrown']
### Add new algorithms here...


#Set up ML algorithm True/False use
if not eval(do_all): #If do all algorithms is false
    for key in algInfo:
        algInfo[key][0] = False #Set algorithm use to False

#Set algorithm use truth for each algorithm specified by user (i.e. if user specified True/False for a specific algorithm)
if not do_linReg == 'None':
    algInfo['Linear Regression'][0] = eval(do_linReg)
if not do_ENReg == 'None':
    algInfo['Elastic Net'][0] = eval(do_ENReg)
if not do_GL == 'None':
    algInfo['Group Lasso'][0] = eval(do_GL)
if not do_L21Reg == 'None':
    algInfo['L21Reg'][0] = eval(do_L21Reg)
if not do_L21GMMReg == 'None':
    algInfo['L21GMMReg'][0] = eval(do_L21GMMReg)
if not do_L21DGMMReg == 'None':
    algInfo['L21DGMMReg'][0] = eval(do_L21DGMMReg)

if not do_RFReg == 'None':
    algInfo['RF Regressor'][0] = eval(do_RFReg)
if not do_AdaReg == 'None':
    algInfo['AdaBoost'][0] = eval(do_AdaReg)
if not do_GradReg == 'None':
    algInfo['GradBoost'][0] = eval(do_GradReg)
if not do_SVR == 'None':
    algInfo['SVR'][0] = eval(do_SVR)
### Add new algorithms here...



#Pickle the algorithm information dictionary for future use
pickle_out = open(output_path+'/'+experiment_name+'/'+"algInfo.pickle", 'wb')
pickle.dump(algInfo,pickle_out)
pickle_out.close()

#Make list of algorithms to be run (full names)
algorithms = []
for key in algInfo:
    if algInfo[key][0]: #Algorithm is true
        algorithms.append(key)

### Run ML Modeling

In [ ]:
dataset_paths = os.listdir(output_path + "/" + experiment_name)
removeList = removeList = ['L21', 'metadata.pickle','metadata.csv','algInfo.pickle','jobsCompleted','logs','jobs','DatasetComparisons','UsefulNotebooks',experiment_name+'_ML_Pipeline_Report.pdf']
for text in removeList:
    if text in dataset_paths:
        dataset_paths.remove(text)
print(dataset_paths)
if do_CommonReg == 'True':
  for dataset_directory_path in dataset_paths:
      full_path = output_path + "/" + experiment_name + "/" + dataset_directory_path
      if not os.path.exists(full_path+'/models'):
          os.mkdir(full_path+'/models')
      if not os.path.exists(full_path+'/model_evaluation'):
          os.mkdir(full_path+'/model_evaluation')
      if not os.path.exists(full_path+'/models/pickledModels'):
          os.mkdir(full_path+'/models/pickledModels')

      for cvCount in range(cv_partitions):
          train_file_path = full_path+'/CVDatasets/'+dataset_directory_path+"_CV_"+str(cvCount)+"_Train.csv"
          test_file_path = full_path + '/CVDatasets/' + dataset_directory_path + "_CV_" + str(cvCount) + "_Test.csv"
          for algorithm in algorithms:
              if algorithm != 'L21Reg' and algorithm !='L21GMMReg' and algorithm !='L21DGMMReg':
                  print(algorithm)
                  algAbrev = algInfo[algorithm][1]
                  #Get header names for current CV dataset for use later in GP tree visulaization
                  data_name = full_path.split('/')[-1]
                  feature_names = pd.read_csv(full_path+'/CVDatasets/'+data_name+'_CV_'+str(cvCount)+'_Test.csv').columns.values.tolist()
                  if instance_label != 'None':
                      feature_names.remove(instance_label)
                  feature_names.remove(class_label)
                  #Get hyperparameter grid
                  param_grid = hyperparameters(random_state,feature_names)[algorithm]
                  ModelJob.runModel(algorithm,train_file_path,test_file_path,full_path,n_trials,timeout,export_hyper_sweep_plots,instance_label,class_label,random_state,cvCount,filter_poor_features,training_subsample,use_uniform_FI,primary_metric,param_grid,groups_path, algAbrev)

if do_L21Series == 'True':
  full_path = output_path + "/" + experiment_name + "/" + 'L21'
  for cvCount in range(cv_partitions):
      dataset_paths_2 = dataset_paths.copy()
      train_file_path_1 = full_path+'/'+dataset_paths[0]+'/CVDatasets/'+dataset_paths_2[0]+"_CV_"+str(cvCount)+"_Train.csv"
      test_file_path_1 = full_path+'/'+dataset_paths[0]+ '/CVDatasets/' + dataset_paths_2[0] + "_CV_" + str(cvCount) + "_Test.csv"
      train_file_path_2 = full_path+'/'+dataset_paths[1]+'/CVDatasets/'+dataset_paths_2[1]+"_CV_"+str(cvCount)+"_Train.csv"
      test_file_path_2 = full_path +'/'+dataset_paths[1]+ '/CVDatasets/' + dataset_paths_2[1] + "_CV_" + str(cvCount) + "_Test.csv"
      train_file_path_3 = full_path+'/'+dataset_paths[2]+'/CVDatasets/'+dataset_paths_2[2]+"_CV_"+str(cvCount)+"_Train.csv"
      test_file_path_3 = full_path +'/'+dataset_paths[2]+ '/CVDatasets/' + dataset_paths_2[2] + "_CV_" + str(cvCount) + "_Test.csv"
      for algorithm in algorithms:
        if algorithm == 'L21Reg' or algorithm =='L21GMMReg' or algorithm =='L21DGMMReg':
            print(algorithm)
            algAbrev = algInfo[algorithm][1]
            #Get header names for current CV dataset for use later in GP tree visulaization
            data_name = full_path.split('/')[-1]
            feature_names = pd.read_csv(full_path+'/'+dataset_paths[0]+'/CVDatasets/'+dataset_paths_2[0]+'_CV_'+str(cvCount)+'_Test.csv').columns.values.tolist()
            if instance_label != 'None':
                feature_names.remove(instance_label)
            feature_names.remove(class_label)
            #Get hyperparameter grid
            param_grid = hyperparameters(random_state,feature_names)[algorithm]
            ModelJob.runModel_2(algorithm,train_file_path_1, train_file_path_2, train_file_path_3,test_file_path_1, test_file_path_2, test_file_path_3,full_path,n_trials,timeout,export_hyper_sweep_plots,instance_label,class_label,random_state,cvCount,filter_poor_features,training_subsample,use_uniform_FI,primary_metric,param_grid,algAbrev, output_path, experiment_name)

#Unpickle metadata from previous phase
file = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()

#Update metadata
### Add new algorithms here...
metadata['Linear Regression'] = str(algInfo['Linear Regression'][0])
metadata['Elastic Net'] = str(algInfo['Elastic Net'][0])
metadata['Group Lasso'] = str(algInfo['Group Lasso'][0])
metadata['L21Reg'] = str(algInfo['L21Reg'][0])
metadata['L21GMMReg'] = str(algInfo['L21GMMReg'][0])
metadata['L21DGMMReg'] = str(algInfo['L21DGMMReg'][0])


metadata['Primary Metric'] = primary_metric
metadata['Training Subsample for KNN,ANN,SVM,and XGB'] = training_subsample
metadata['Uniform Feature Importance Estimation (Models)'] = use_uniform_FI
metadata['Hyperparameter Sweep Number of Trials'] = n_trials
metadata['Hyperparameter Sweep Number of Trials'] = n_trials
metadata['Hyperparameter Timeout'] = timeout
metadata['Export Hyperparameter Sweep Plots'] = export_hyper_sweep_plots

#Pickle the metadata for future use
pickle_out = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'wb')
pickle.dump(metadata,pickle_out)
pickle_out.close()

## -----------------------------------------------------------------------------------------------------------------
## Phase 6: Statistics (Stats Summaries, Figures, Statistical Comparisons)

### Import Additional Python Packages

In [ ]:
import StatsJob

### Run Statistics Summary and Figure Generation

In [ ]:
#Unpickle metadata from previous phase
file = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()
metadata['Export Metric Boxplots'] = plot_metric_boxplots
metadata['Export Feature Importance Boxplots'] = plot_FI_box
metadata['Metric Weighting Composite FI Plots'] = metric_weight
metadata['Top Model Features To Display'] = top_model_features
#Pickle the metadata for future use
pickle_out = open(output_path+'/'+experiment_name+'/'+"metadata.pickle", 'wb')
pickle.dump(metadata,pickle_out)
pickle_out.close()

#Now that primary pipeline phases are complete generate a human readable version of metadata
df = pd.DataFrame.from_dict(metadata, orient ='index')
df.to_csv(output_path+'/'+experiment_name+'/'+'metadata.csv',index=True)

# Iterate through datasets
dataset_paths = os.listdir(output_path + "/" + experiment_name)
removeList = removeList = ['metadata.pickle','metadata.csv','algInfo.pickle','jobsCompleted','logs','jobs','DatasetComparisons','UsefulNotebooks',experiment_name+'_ML_Pipeline_Report.pdf']
for text in removeList:
    if text in dataset_paths:
        dataset_paths.remove(text)
for dataset_directory_path in dataset_paths:
    if dataset_directory_path != 'L21':
        full_path = output_path + "/" + experiment_name + "/" + dataset_directory_path
        StatsJob.job(full_path,plot_FI_box,class_label,instance_label,cv_partitions,scale_data,plot_metric_boxplots,primary_metric,top_model_features,sig_cutoff,metric_weight,jupyterRun)


## -----------------------------------------------------------------------------------------------------------------
## Phase 7: Dataset Comparison (Optional: Use only if > 1 dataset was analyzed)

### Import Additional Python Packages

In [ ]:
import DataCompareJob

### Run Dataset Comparison

In [ ]:
if do_L21Series == 'True':
    dataset_paths.remove('L21')
if len(dataset_paths) > 1:
    DataCompareJob.job(output_path+'/'+experiment_name,sig_cutoff,jupyterRun)

## -----------------------------------------------------------------------------------------------------------------
## Phase 8: PDF Training Report Generator (Optional)

In [ ]:
import PDF_ReportJob_Reg

In [ ]:
experiment_path = output_path+'/'+experiment_name
PDF_ReportJob_Reg.job(experiment_path,'True','None','None')

## -----------------------------------------------------------------------------------------------------------------
## Phase 9: Apply Models to Replication Data (Optional)

### Import Additional Python Packages

In [ ]:
import ApplyModelJob

### Specify Run Parameters

In [ ]:
if demo_run:
    rep_data_path = wd_path+'/drive/MyDrive/STREAMLINE-main/DemoRepData'
print("Replication Data Folder Path: "+rep_data_path)
print("Dataset Path: "+dataset_for_rep)

### Run Application of Models to Replication Data

In [ ]:
if applyToReplication:
    data_name = dataset_for_rep.split('/')[-1].split('.')[0] #Save unique dataset names so that analysis is run only once if there is both a .txt and .csv version of dataset with same name.
    full_path = output_path + "/" + experiment_name + "/" + data_name #location of folder containing models respective training dataset
    full_path
    # full_path_2 = output_path + "/" + experiment_name + "/" + data_name
    if not os.path.exists(full_path):
        os.mkdir(full_path)
    if not os.path.exists(full_path+"/applymodel"):
        os.mkdir(full_path+"/applymodel")

    #Determine file extension of datasets in target folder:
    file_count = 0
    unique_datanames = []
    for datasetFilename in glob.glob(rep_data_path+'/*'):
        datasetFilename = str(datasetFilename).replace('\\','/')

        file_extension = datasetFilename.split('/')[-1].split('.')[-1]
        apply_name = datasetFilename.split('/')[-1].split('.')[0] #Save unique dataset names so that analysis is run only once if there is both a .txt and .csv version of dataset with same name.
        if not os.path.exists(full_path+"/applymodel/"+apply_name):
            os.mkdir(full_path+"/applymodel/"+apply_name)

        if file_extension == 'txt' or file_extension == 'csv':
            if apply_name not in unique_datanames:
                unique_datanames.append(apply_name)
                ApplyModelJob.job(datasetFilename,full_path,class_label,instance_label,categorical_cutoff,sig_cutoff,cv_partitions,scale_data,impute_data,primary_metric,dataset_for_rep,match_label,plot_ROC,plot_PRC,plot_metric_boxplots,export_feature_correlations,jupyterRun,multi_impute)
                file_count += 1

    if file_count == 0: #Check that there was at least 1 dataset
        raise Exception("There must be at least one .txt or .csv dataset in rep_data_path directory")

## -----------------------------------------------------------------------------------------------------------------
## Phase 10: PDF Apply Report Generator (Optional)

In [ ]:
import PDF_ReportJob

In [ ]:
if applyToReplication:
    experiment_path = output_path+'/'+experiment_name
    PDF_ReportJob.job(experiment_path,'False',rep_data_path,dataset_for_rep)

## -----------------------------------------------------------------------------------------------------------------
## Phase 11: File Cleanup (Optional)

In [ ]:
# Get dataset paths for all completed dataset analyses in experiment folder
datasets = os.listdir(experiment_path)
experiment_name = experiment_path.split('/')[-1] #Name of experiment folder
removeList = removeList = ['metadata.pickle','metadata.csv','algInfo.pickle','jobsCompleted','logs','jobs','DatasetComparisons','UsefulNotebooks',experiment_name+'_ML_Pipeline_Report.pdf']
for text in removeList:
    if text in datasets:
        datasets.remove(text)

#Delete jobscompleted folder/files
try:
    shutil.rmtree(experiment_path+'/'+'jobsCompleted')
except:
    pass

#Delete target files within each dataset subfolder
for dataset in datasets:
    #Delete individual runtime files (save runtime summary generated in phase 6)
    if eval(del_time):
        try:
            shutil.rmtree(experiment_path+'/'+dataset+'/'+'runtime')
            print("Individual Runtime Files Deleted")
        except:
            pass
    #Delete temporary feature importance pickle files (only needed for phase 4 and then saved as summary files in phase 6)
    try:
        shutil.rmtree(experiment_path+'/'+dataset+'/feature_selection/mutualinformation/pickledForPhase4')
        print("Mutual Information Pickle Files Deleted")
    except:
        pass
    try:
        shutil.rmtree(experiment_path+'/'+dataset+'/feature_selection/multisurf/pickledForPhase4')
        print("MultiSURF Pickle Files Deleted")
    except:
        pass
    #Delete older training and testing CV datasets (does not delete any final versions used for training). Older cv datasets might have been kept to see what they look like prior to preprocessing and feature selection.
    if eval(del_oldCV):
        #Delete CV files generated after preprocessing but before feature selection
        files = glob.glob(experiment_path+'/'+dataset+'/CVDatasets/*CVOnly*')
        for f in files:
            try:
                os.remove(f)
                print("Deleted Intermediary CV-Only Dataset Files")
            except:
                pass
        #Delete CV files generated after CV partitioning but before preprocessing
        files = glob.glob(experiment_path+'/'+dataset+'/CVDatasets/*CVPre*')
        for f in files:
            try:
                os.remove(f)
                print("Deleted Intermediary CV-Pre Dataset Files")
            except:
                pass